In [98]:
import re
import json


def parse_name(name):
    """Takes a name string in any format and returns the first name, last name, and middle initial."""
    # Remove leading/trailing whitespace
    name = name.strip()

    # Define regular expression patterns for different name formats
    patterns = [
        r'^(?P<first>\w+)\s+(?P<middle>\w\.)\s+(?P<last>\w+)$',  # Firstname M. Lastname
        r'^(?P<first>\w+)\s+(?P<last>\w+)$',                     # Firstname Lastname
        r'^(?P<last>\w+),\s+(?P<first>\w+)$',                    # Lastname, Firstname
        r'^(?P<last>\w+),\s+(?P<first>\w+)\s+(?P<middle>\w\.)$'  # Lastname, Firstname M.
    ]

    # Iterate over the patterns and try to match the name
    for pattern in patterns:
        match = re.match(pattern, name)
        if match:
            # Extract the matched groups
            first = match.group('first')
            last = match.group('last')
            middle = match.group('middle') if 'middle' in match.groupdict() else ''

            # Capitalize the first letter and lowercase the rest
            first = first.capitalize()
            last = last.capitalize()
            middle = middle.capitalize() if middle else None

            return first, last, middle

    # If no match is found, return None for all fields
    return None, None, None


class Group:
    """Class to store a group of Person objects and the types of relationships among them."""
    def __init__(self, filename=None):
        self.relationships = dict()
        self.people = dict()
        if filename:
            load_people_from_file(self, filename)

    def save_group_to_file(self, filename):
        data = {
            'relationships': self.relationships,
            'people': {
                fullname: {
                    attr: getattr(person, attr)
                    for attr in dir(person)
                    if not attr.startswith('__') and not callable(getattr(person, attr)) and attr != 'group'
                }
                for fullname, person in self.people.items()
            }
        }
        with open(filename, 'w') as file:
            json.dump(data, file, indent=4)


class Person:
    """Class to represent a person and list their relationships."""
    def __init__(self, name, group):
        """A Person is initialized using a name and the Group they belong to."""
        # Parse name
        self.firstname, self.lastname, self.middle = parse_name(name)
        assert self.firstname and self.lastname, "Person must be initialized with a valid full name."
        self.fullname = ' '.join(filter(None, [self.firstname, self.middle, self.lastname]))

        assert isinstance(group, Group), "Person must be initialized with a valid Group." 
        if self.fullname not in group.people:
            group.people[self.fullname] = self
        self.group = group


    def add_undirected_relationship(self, name, relationship):
        """Add a mutual relationship to the Person with specified name (such as 'friends')."""
        if relationship not in self.group.relationships:
            self.group.relationships[relationship] = 'undirected'
        else:
            assert self.group.relationships[relationship] == 'undirected', f'Relationship "{relationship}" is already saved as directed'
        target = Person(name, self.group)
        target_name = target.fullname
        if target_name in self.group.people:
            target = self.group.people[target_name]
        if not hasattr(self, relationship):
            setattr(self, relationship, [])
        if target_name not in getattr(self, relationship):
            getattr(self, relationship).append(target_name)
            target.add_undirected_relationship(self.fullname, relationship)


    def add_directed_relationship(self, name, relationship):
        """Add a directed relationship to Person 'name' (such as 'children')."""
        if relationship not in self.group.relationships:
            self.group.relationships[relationship] = 'directed'
        else:
            assert self.group.relationships[relationship] == 'directed', f'Relationship "{relationship}" is already saved as undirected'
        target = Person(name, self.group)
        target_name = target.fullname
        if target_name in self.group.people:
            target = self.group.people[target_name]
        if not hasattr(self, relationship):
            setattr(self, relationship, [])
        if target_name not in getattr(self, relationship):
            getattr(self, relationship).append(target_name)


def load_people_from_file(group, filename):
    """Load Person objects from saved json file into Group object."""
    try:
        with open(filename, 'r') as file:
            data = json.load(file)
            relationships = data.get('relationships', {})
            for relationship, kind in relationships.items():
                if relationship in group.relationships:
                    assert kind == group.relationships[relationship], f'Conflicting relationship: existing relationship "{relationship}" is {group.relationships[relationship]}, while new relationship "{relationship}" is {kind}'
                else:
                    group.relationships[relationship] = kind
            for fullname, attrs in data.get('people', {}).items():
                person = Person(fullname, group)
                for attr, values in attrs.items():
                    setattr(person, attr, values)
    except FileNotFoundError:
        print(f"File '{filename}' not found.")

In [99]:
# Example usage
group = Group()
john = Person('John Doe', group)
jane = Person('Jane Smith', group)
john.add_undirected_relationship('Jane Smith', 'friends')
john.add_directed_relationship('Hank Doe', 'children')
print(group.relationships, group.people)
save_people_to_file(group, 'test_people.json')
group2 = Group('test_people.json')
print(group2.relationships, group2.people)

{'friends': 'undirected', 'children': 'directed'} {'John Doe': <__main__.Person object at 0x10cc9b1d0>, 'Jane Smith': <__main__.Person object at 0x108297470>, 'Hank Doe': <__main__.Person object at 0x10cc9b0b0>}
{'friends': 'undirected', 'children': 'directed'} {'John Doe': <__main__.Person object at 0x10cc9b830>, 'Jane Smith': <__main__.Person object at 0x10cc98110>, 'Hank Doe': <__main__.Person object at 0x10cc9b740>}
